In [3]:
import io
import glob
import os
from shutil import move
from os.path import join
from os import listdir, rmdir
import cv2
import numpy as np
import torch
import glob
import pickle
import torch.nn as nn
import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from google.colab import files
import time
import struct
import PIL
import torchvision.models as models
from google.colab import drive
drive.mount('/content/drive')
!cp "drive/My Drive/DL-finalproj/model_eval.py" .

from model_eval import trainer
gpu_boole = torch.cuda.is_available()
print(gpu_boole)

def getTinyImageNet():
  !wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
  !unzip tiny-imagenet-200.zip
  target_folder = './tiny-imagenet-200/val/'
  test_folder   = './tiny-imagenet-200/test/'

  #os.mkdir(test_folder)
  val_dict = {}
  with open('./tiny-imagenet-200/val/val_annotations.txt', 'r') as f:
      for line in f.readlines():
          split_line = line.split('\t')
          val_dict[split_line[0]] = split_line[1]

  paths = glob.glob('./tiny-imagenet-200/val/images/*')
  for path in paths:
      file = path.split('/')[-1]
      folder = val_dict[file]
      if not os.path.exists(target_folder + str(folder)):
          os.mkdir(target_folder + str(folder))
          os.mkdir(target_folder + str(folder) + '/images')
      if not os.path.exists(test_folder + str(folder)):
          os.mkdir(test_folder + str(folder))
          os.mkdir(test_folder + str(folder) + '/images')


  for path in paths:
      file = path.split('/')[-1]
      folder = val_dict[file]
      if len(glob.glob(target_folder + str(folder) + '/images/*')) <25:
          dest = target_folder + str(folder) + '/images/' + str(file)
      else:
          dest = test_folder + str(folder) + '/images/' + str(file)
      move(path, dest)

  rmdir('./tiny-imagenet-200/val/images')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
True


In [ ]:
getTinyImageNet() #run once only

In [5]:
def AugmentedImageNet(train_transforms=[], val_transforms=[], test_transforms=[], batch_size=100):
  
    data_transforms = {
        'train': transforms.Compose(
            train_transforms
        ),
        'val': transforms.Compose(
            val_transforms
        ),
        'test': transforms.Compose(
            test_transforms
        )
    }

    data_dir = 'tiny-imagenet-200/'
    num_workers = {
        'train' : 100,
        'val'   : 0,
        'test'  : 0
    }
    image_datasets = {x: dataset.ImageFolder(os.path.join(data_dir, x),
                                              data_transforms[x])
                      for x in ['train', 'val','test']}
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=100,
                                                 shuffle=True, pin_memory =True)
                  for x in ['train', 'val', 'test']}

    return dataloaders

In [6]:
epochs = 10
netparam = models.googlenet(pretrained=True)
netparam.fc.out_features = 200
for param in netparam.parameters():
  p
optimizer = torch.optim.Adam(netparam.parameters(), 0.001)
loss_metric = torch.nn.CrossEntropyLoss()
if gpu_boole:
 netparam = netparam.cuda()




train_model =  trainer(netparam, dataloaders, optimizer, loss_metric, 
                                epochs, gpu_boole, 1, 1, 1, 0)



Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


In [10]:
baseline_data_dict = {
    
}

augmented_data_dict = {0 : AugmentedImageNet(train_transforms=[  
      transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter()]), p=0.1),   
      transforms.ToTensor(),                                                                             
      transforms.RandomGrayscale(0.1),
      transforms.RandomApply(torch.nn.ModuleList([transforms.RandomRotation(10)]), p=0.1),
      transforms.RandomErasing(p=0.1)], 
    val_transforms=[
      transforms.ToTensor(),
      transforms.GaussianBlur(3, sigma=1.0)
    ],
    test_transforms=[]),

    1 : ,

    2 : , 

    3 : ,

    4 : , 
}

In [ ]:
model_name = 'GoogleNet_'
for i in range(5):
  netparam = models.googlenet(pretrained=True)
  netparam.fc.out_features = 200
  # FREEZE ALL BUT LAST LAYER
  optimizer = torch.optim.Adam(netparam.parameters(), 0.001)
  loss_metric = torch.nn.CrossEntropyLoss()
  if gpu_boole:
  netparam = netparam.cuda()

  
  curr_baseline_data = baseline_data_dict[i]
  curr_aug_data = augmented_data_dict[i]



  baseline_trainer =  trainer(netparam, curr_baseline_data, optimizer, loss_metric, 
                                  epochs=25, gpu_boole, 1, 1, 1, 0)

  bl_train_acc, bl_epoch_loss, _, bl_validation_acc,_ = baseline_trainer.train_model()

  bl_results_pickle = {'train_acc': bl_train_acc,
                       'epoch_loss': bl_epoch_loss,
                       'val_acc': bl_validation_acc}
  

  pickle.dump(bl_results_pickle, open('bl_results_' + model_name + str(i) +  '.pkl', 'wb'))
  torch.save(netparam.state_dict(), 'models/baseline/' + model_name + str(i) + '.model')


  # UNFREEZE ALL LAYERS

  
  augmented_trainer = trainer(netparam, curr_augmented_data, optimizer, loss_metric, 
                                  epochs=25, gpu_boole, 1, 1, 1, 0)


  aug_train_acc, aug_epoch_loss, _, aug_validation_acc,_ = augmented_trainer.train_model()
  aug_results_pickle = {'train_acc': aug_train_acc,
                       'epoch_loss': aug_epoch_loss,
                       'val_acc': aug_validation_acc}
  pickle.dumb(aug_results_pickle, open('aug_results_' + model_name + str(i) +  '.pkl', 'wb'))
  torch.save(netparam.state_dict(), 'models/augmented/' + model_name + str(i) + '.model')

torch.Size([100, 3, 64, 64]) torch.Size([100])


In [17]:
netparam.parameters()

<generator object Module.parameters at 0x7f683c5a7d00>
